In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import log_loss
from nltk import word_tokenize
from nltk import FreqDist
from nltk.corpus import stopwords
import string
import seaborn as sns
import matplotlib.pyplot as plt
from geopy.distance import vincenty

In [ ]:
df = pd.read_json(open("../input/train.json","r"))

In [55]:
df["created"] = pd.to_datetime(df["created"])
df["time_since_built"] = pd.datetime.today() - df["created"]
df["days_since_built"] = df["time_since_built"].astype('timedelta64[D]').astype(int)
df["len_descrip"] = df["description"].apply(lambda x: len(x.split(" ")))
# set bad words for features such as pre-war, prewar. turns out prewar is a positive word
# bad_words = ['Pre-War','PreWar','Prewar','prewar','pre-war','pre war','Pre War'] lambda x: len([item for item in x if item not in bad_words])
df["len_features"] = df["features"].apply(len)
df["num_photos"] = df["photos"].apply(len)

In [22]:
all_descrip = list(df["description"])
all_descrip_words = ' '.join(all_descrip)
all_feat = []
all_feat.extend(df["features"])
# all_feat_sum = sum(all_feat,[]) this code works but is tooooo slow, the following code is way better, 10 times faster
all_feat_sum = [item.lower() for sublist in all_feat for item in sublist] # lowercase reduced freq from 1595 to 1294
print(len(all_descrip_words),len(all_feat_sum))

In [97]:
freq = FreqDist(all_feat_sum)
freq_df = pd.DataFrame(freq.most_common(len(freq)))
freq_df.columns = [['features','frequencies']]
freq_df.head()

In [124]:
high_freq_feat = list(freq_df[:int(len(freq)/3)]["features"])
med_freq_feat = list(freq_df[int(len(freq)/3):int(len(freq)*2/3)]["features"])
low_freq_feat = list(freq_df[int(len(freq)*2/3):]["features"])
print(high_freq_feat[:10],med_freq_feat[:10],low_freq_feat[:10])

In [126]:
df["num_high_feat"] = df["features"].apply(lambda x: len([item for item in x if item.lower() in high_freq_feat]))
df["num_med_feat"] = df["features"].apply(lambda x: len([item for item in x if item.lower() in med_freq_feat]))
df["num_low_feat"] = df["features"].apply(lambda x: len([item for item in x if item.lower() in low_freq_feat]))
df.head()

In [28]:
sns.barplot(x='features', y = 'frequencies', data = freq_df)
plt.xticks(rotation=60, fontsize = 10)

In [43]:
df["num_badwords"] = df["features"].apply(lambda x: len([item for item in x if item in bad_words]))

In [ ]:
stop = stopwords.words('english') + list(string.punctuation)
stop.extend(['br','br/','p','1','2','3','--',"''","'s"])
tokens = word_tokenize(all_descrip_words.lower())
clean_descrip = [i for i in tokens if i not in stop]
freq = FreqDist(clean_descrip)

In [ ]:
n = 50
freq_df = pd.DataFrame(freq.most_common(n))
freq_df.columns = [['word','frequencies']]
freq_df.head()

In [ ]:
sns.barplot(x='word', y = 'frequencies', data = freq_df)
plt.xticks(rotation=60, fontsize = 10)

In [48]:
# set keywords according to the frequencies of all words
key_words = ['new','appliances','renovated','large','great','spacious','beautiful','beautifully','brand','close','top','excellent','recently','luxury','clean','nice','big','kitchen','amenity','safe']
df["num_keywords"] = df["description"].apply(lambda x: sum(x.count(i) for i in key_words))

In [45]:
# calculate the distance from time square and downtown world trade center
time_square = [40.7589, -73.9851]
world_trade_center = [40.7124, -74.0128]
df["location"] = list(zip(df["latitude"],df["longitude"]))
df["dist_ts"] = df["location"].apply(lambda x: vincenty(time_square, x).miles)
df["dist_wtc"] = df["location"].apply(lambda x: vincenty(world_trade_center, x).miles)

In [85]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [137]:
num_feats = ["bedrooms",  "dist_wtc", "price",
             "num_photos", "len_descrip", "num_keywords", "num_high_feat","num_med_feat","num_low_feat",
             "days_since_built"] #remove "dist_ts",  "len_features", "bathrooms", "num_badwords",
#X = df[num_feats]
X = pd.DataFrame(scaler.fit_transform(df[num_feats]), columns=num_feats)
y = df["interest_level"]
X.head()

In [130]:
corrmat = X.corr()
sns.heatmap(corrmat, vmax=.8, square=True)

In [138]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.33)

In [139]:
clf = RandomForestClassifier(n_estimators=1000)
clf.fit(X_train, y_train)
y_val_pred = clf.predict_proba(X_val)
log_loss(y_val, y_val_pred)

In [ ]:
clf.score(X_val,y_val)

In [ ]:
df = pd.read_json(open("../input/test.json", "r"))
df["num_photos"] = df["photos"].apply(len)
df["num_features"] = df["features"].apply(len)
df["num_description_words"] = df["description"].apply(lambda x: len(x.split(" ")))
df["created"] = pd.to_datetime(df["created"])
df["created_year"] = df["created"].dt.year
df["created_month"] = df["created"].dt.month
df["created_day"] = df["created"].dt.day
X = df[num_feats]

y = clf.predict_proba(X)

In [ ]:
labels2idx = {label: i for i, label in enumerate(clf.classes_)}
labels2idx

In [ ]:
sub = pd.DataFrame()
sub["listing_id"] = df["listing_id"]
for label in ["high", "medium", "low"]:
    sub[label] = y[:, labels2idx[label]]
sub.to_csv("submission_rf.csv", index=False)

In [ ]:
'''from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
gp_fix = GaussianProcessClassifier(kernel=1.0 * RBF(length_scale=1.0), optimizer=None)
gp_fix.fit(X_train, y_train)
y_val_pred = gp_fix.predict_proba(X_val)
log_loss(y_val,y_val_pred)
gp_opt = GaussianProcessClassifier(kernel=1.0 * RBF(length_scale=1.0))
gp_opt.fit(X_train, y_train)''''''

In [142]:
''''C = 1.0  # SVM regularization parameter
svc = svm.SVC(kernel='linear', C=C, probability=True)
svc.fit(X_train, y_train)
y_val_pred = svc.predict_proba(X_val)
log_loss(y_val,y_val_pred) #0.76626587505159971''''

In [143]:
''''rbf_svc = svm.SVC(kernel='rbf', gamma=0.7, C=C, probability=True)
rbf_svc.fit(X_train, y_train)
y_val_pred = rbf_svc.predict_proba(X_val)
log_loss(y_val,y_val_pred) #0.78343422893897285 very slow and worse than RF''''

In [144]:
''''poly_svc = svm.SVC(kernel='poly', degree=3, C=C, probability=True)
poly_svc.fit(X_train, y_train)
y_val_pred = poly_svc.predict_proba(X_val)
log_loss(y_val,y_val_pred) #0.78747043654211291''''

In [147]:
from sklearn.ensemble import GradientBoostingClassifier
gbc = GradientBoostingClassifier(n_estimators=1000)
gbc.fit(X_train, y_train)
y_val_pred = gbc.predict_proba(X_val)
log_loss(y_val, y_val_pred) #0.64473894356754169 for n_estimators = 100 0.63055798833692045 for n_estimators = 1000

In [149]:
from sklearn.ensemble import AdaBoostClassifier
abc = AdaBoostClassifier(n_estimators=1000)
abc.fit(X_train, y_train)
y_val_pred = abc.predict_proba(X_val)
log_loss(y_val, y_val_pred) #1.0844319221850884 for n-estimators = 100 1.0970218291172218 for n-estimators=1000

In [152]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier(max_features=10)
dtc.fit(X_train, y_train)
y_val_pred = dtc.predict_proba(X_val)
log_loss(y_val, y_val_pred) #12.550469062345963 12.47179278909033 for max_features=10

In [155]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(100, 2), random_state=1)
clf.fit(X_train, y_train)
y_val_pred = clf.predict_proba(X_val)
log_loss(y_val, y_val_pred) #0.7796157421558173

0.76233173622689721